In [7]:
def level_data(data):
    # 2D polynomial leveling (3rd degree)
    x = np.linspace(-1, 1, data.shape[1])
    y = np.linspace(-1, 1, data.shape[0])
    X, Y = np.meshgrid(x, y)
    X_flat = X.flatten()
    Y_flat = Y.flatten()
    Z_flat = data.flatten()

    # Construct polynomial terms up to 3rd degree
    A = np.column_stack([
        np.ones_like(X_flat), X_flat, Y_flat, 
        X_flat**2, X_flat*Y_flat, Y_flat**2,
        X_flat**3, (X_flat**2)*Y_flat, X_flat*(Y_flat**2), Y_flat**3
    ])
    coeffs, *_ = np.linalg.lstsq(A, Z_flat, rcond=None)
    Z_fit = A @ coeffs
    Z_fit_reshaped = Z_fit.reshape(data.shape)

    return data - Z_fit_reshaped

In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from pySPM import Bruker
from PIL import Image
from tqdm import tqdm

def gaussian(x, a, mu, sigma):
    return a * np.exp(-(x - mu) ** 2 / (2 * sigma ** 2))

def level_data(data):
    # 2D polynomial leveling (2nd degree)
    x = np.linspace(-1, 1, data.shape[1])
    y = np.linspace(-1, 1, data.shape[0])
    X, Y = np.meshgrid(x, y)
    X_flat = X.flatten()
    Y_flat = Y.flatten()
    Z_flat = data.flatten()

    # Construct polynomial terms up to 2nd degree
    A = np.column_stack([np.ones_like(X_flat), X_flat, Y_flat, X_flat**2, X_flat*Y_flat, Y_flat**2])
    coeffs, *_ = np.linalg.lstsq(A, Z_flat, rcond=None)
    Z_fit = A @ coeffs
    Z_fit_reshaped = Z_fit.reshape(data.shape)

    return data - Z_fit_reshaped



In [13]:
def align_rows(data):
    # Align rows using median difference
    for i in range(1, data.shape[0]):
        offset = np.median(data[i, :] - data[i - 1, :])
        data[i, :] -= offset
    return data

def adjust_contrast(data):
    # Fit Gaussian to histogram
    flattened = data.flatten()
    hist, bin_edges = np.histogram(flattened, bins=256)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Initial guess
    p0 = [np.max(hist), np.mean(flattened), np.std(flattened)]
    try:
        popt, _ = curve_fit(gaussian, bin_centers, hist, p0=p0)
        _, mu, sigma = popt
    except RuntimeError:
        mu, sigma = np.mean(flattened), np.std(flattened)

    vmin = mu - 6 * sigma
    vmax = mu + 6 * sigma
    return np.clip((data - vmin) / (vmax - vmin), 0, 1)

def save_as_16bit_png(data, filename):
    data_16bit = (data * 65535).astype(np.uint16)
    img = Image.fromarray(data_16bit, mode='I;16')
    img.save(filename)

def process_sur_file(filepath, output_folder):
    print(f"Processing file: {filepath}")
    spm = Bruker(filepath)
    image = spm.get_channel("Height")  # assume first channel is height
    data = image.pixels.copy()

    data = level_data(data)
    data = align_rows(data)
    data_norm = adjust_contrast(data)

    base_name = os.path.splitext(os.path.basename(filepath))[0]
    output_path = os.path.join(output_folder, base_name + '.png')
    save_as_16bit_png(data_norm, output_path)

def process_folder(folder):
    output_folder = os.path.join(folder, "processed_pngs")
    os.makedirs(output_folder, exist_ok=True)

    sur_files = [f for f in os.listdir(folder) if f.lower().endswith('.spm')]
    for sur_file in tqdm(sur_files, desc="Processing .spm files"):
        filepath = os.path.join(folder, sur_file)
        process_sur_file(filepath, output_folder)


In [14]:

if __name__ == "__main__":
    # import argparse

    # parser = argparse.ArgumentParser(description="Process Bruker AFM .sur files with pySPM.")
    # parser.add_argument("folder", help="Folder containing .sur files")
    # args = parser.parse_args()

    # process_folder(args.folder)
    folder = r"C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025"
    process_folder(folder)


Processing .spm files:   9%|▉         | 1/11 [00:00<00:01,  8.91it/s]

Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_01.0_00000.spm
Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_02.0_00000.spm


Processing .spm files:  27%|██▋       | 3/11 [00:00<00:01,  7.86it/s]

Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_03.0_00000.spm
Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_04.0_00000.spm


Processing .spm files:  45%|████▌     | 5/11 [00:00<00:00,  8.95it/s]

...

Processing .spm files:  73%|███████▎  | 8/11 [00:00<00:00,  9.70it/s]

Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_07.0_00000.spm
Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_08.0_00000.spm


Processing .spm files:  91%|█████████ | 10/11 [00:01<00:00, 10.27it/s]

Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_09.0_00000.spm
Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_10.0_00000.spm
Processing file: C:\Users\cobia\OneDrive - University of Cambridge\AFM\09_07_2025\short_end_11.0_00000.spm


Processing .spm files: 100%|██████████| 11/11 [00:01<00:00,  9.47it/s]
